In [24]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

import os
from pathlib import Path
from collections import Counter


In [ ]:
cats_dogs_dir = "data/cats_dogs/PetImages/"
food_101_dir = "data/food-101/food-101/"
food101_image_dir = food_101_dir + "images/"
food101_meta_dir = food_101_dir + "meta/"

## FOOD 101 PROCESSING

In [26]:
with open(food101_meta_dir + 'classes.txt') as f:
    food_classes = [line.rstrip() for line in f]

In [27]:
# Data cleaning: Check for corrupted images and count sizes of images.
for food_cls in food_classes:
    corrupted_images = []
    sizes = []
    image_cls_dir = food101_image_dir + food_cls + '/'
    print(f"analyze folder {image_cls_dir}")

    for filename in os.listdir(image_cls_dir):
        path = os.path.join(image_cls_dir, filename)
        img = cv2.imread(path)
        if img is None:
            corrupted_images.append(filename)
            continue
        sizes.append(img.shape[:2])
    print(Counter(sizes))
    print(f'corrupted images: {len(corrupted_images)}')
    print("-" * 30)


analyze folder data/food-101/food-101/images/apple_pie/
Counter({(512, 512): 632, (384, 512): 164, (512, 384): 51, (341, 512): 14, (382, 512): 14, (512, 382): 14, (288, 512): 8, (512, 511): 7, (342, 512): 7, (340, 512): 6, (306, 512): 6, (383, 512): 6, (511, 512): 5, (289, 512): 5, (343, 512): 5, (307, 512): 4, (512, 308): 3, (512, 340): 2, (512, 289): 2, (510, 512): 2, (358, 512): 2, (301, 512): 2, (512, 341): 2, (385, 512): 2, (512, 306): 2, (287, 512): 1, (503, 512): 1, (481, 512): 1, (474, 512): 1, (313, 512): 1, (326, 512): 1, (442, 512): 1, (349, 512): 1, (512, 339): 1, (317, 512): 1, (512, 404): 1, (256, 512): 1, (512, 509): 1, (479, 512): 1, (399, 512): 1, (512, 426): 1, (459, 512): 1, (483, 512): 1, (472, 512): 1, (512, 379): 1, (512, 480): 1, (369, 512): 1, (487, 512): 1, (512, 487): 1, (394, 512): 1, (339, 512): 1, (232, 512): 1, (322, 512): 1, (506, 512): 1, (512, 288): 1, (477, 512): 1, (417, 512): 1, (480, 512): 1})
corrupted images: 0
------------------------------
analy

## Feature Extraction with DenseNet and DinoV2

In [31]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# Check if GPU is available
def get_device():
    if torch.cuda.is_available():
        return 'cuda'
    elif torch.backends.mps.is_available():
        return 'mps'
    return 'cpu'
device = get_device()
print(f"Using device: {device}")

Using device: mps


In [ ]:
# Initialize DenseNet121 model (pretrained on ImageNet)
densenet_model = models.densenet121(pretrained=True)
# Remove the classification layer to get feature vectors
densenet_model = nn.Sequential(*list(densenet_model.children())[:-1])
densenet_model = densenet_model.to(device)
densenet_model.eval()

densenet_transform = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("DenseNet121 model loaded successfully")
print(f"DenseNet feature vector size: 1024")

/Users/nhanncv/miniforge3/envs/scc451/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/nhanncv/miniforge3/envs/scc451/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DenseNet121 model loaded successfully
DenseNet feature vector size: 1024


In [33]:
dinov2_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
dinov2_model = dinov2_model.to(device)
dinov2_model.eval()

dinov2_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("DinoV2 model loaded successfully")
print(f"DinoV2 feature vector size: 384 (for vits14)")

Using cache found in /Users/nhanncv/.cache/torch/hub/facebookresearch_dinov2_main


DinoV2 model loaded successfully
DinoV2 feature vector size: 384 (for vits14)


In [34]:
def extract_densenet_features(image_path, model, transform, device):
    """Extract feature vector from an image using DenseNet"""
    img = Image.open(image_path)
    img_tensor = transform(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        features = model(img_tensor)
        # Apply adaptive pooling and flatten
        features = nn.functional.adaptive_avg_pool2d(features, (1, 1))
        features = features.view(features.size(0), -1)
    
    return features.cpu().numpy().flatten()

def extract_dinov2_features(image_path, model, transform, device):
    """Extract feature vector from an image using DinoV2"""
    img = Image.open(image_path)
    img_tensor = transform(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        features = model(img_tensor)
    
    return features.cpu().numpy().flatten()

print("Feature extraction functions defined")

Feature extraction functions defined


In [ ]:
# Extract features for Food101 dataset (using original RGB images, not preprocessed ones)
# We'll extract features from a subset of classes for demonstration
# Change this to food_classes if you want to process all 101 classes

densenet_features_dict = {}
dinov2_features_dict = {}
labels = []


for food_cls in food_classes:
    print(f"\nProcessing class: {food_cls}")
    class_dir = Path(food101_image_dir) / food_cls
    image_files = sorted([p for p in class_dir.iterdir() if p.is_file()])
    
    densenet_features_list = []
    dinov2_features_list = []
    
    for img_path in image_files:
        try:
            # Extract DenseNet features
            densenet_feat = extract_densenet_features(img_path, densenet_model, densenet_transform, device)
            densenet_features_list.append(densenet_feat)
            
            # Extract DinoV2 features
            dinov2_feat = extract_dinov2_features(img_path, dinov2_model, dinov2_transform, device)
            dinov2_features_list.append(dinov2_feat)
            
            labels.append(food_cls)
        except Exception as e:
            print(f"    Error processing {img_path.name}: {e}")
            continue
    
    densenet_features_dict[food_cls] = np.array(densenet_features_list)
    dinov2_features_dict[food_cls] = np.array(dinov2_features_list)
    
    print(f"  Extracted {len(densenet_features_list)} feature vectors")

print("\nFeature extraction complete!")


Processing class: apple_pie
  Extracted 1000 feature vectors

Processing class: baby_back_ribs
  Extracted 1000 feature vectors

Processing class: baklava
  Extracted 1000 feature vectors

Processing class: beef_carpaccio
  Extracted 1000 feature vectors

Processing class: beef_tartare
  Extracted 1000 feature vectors

Processing class: beet_salad
  Extracted 1000 feature vectors

Processing class: beignets
  Extracted 1000 feature vectors

Processing class: bibimbap
  Extracted 1000 feature vectors

Processing class: bread_pudding
  Extracted 1000 feature vectors

Processing class: breakfast_burrito
  Extracted 1000 feature vectors

Processing class: bruschetta
  Extracted 1000 feature vectors

Processing class: caesar_salad
  Extracted 1000 feature vectors

Processing class: cannoli
  Extracted 1000 feature vectors

Processing class: caprese_salad
  Extracted 1000 feature vectors

Processing class: carrot_cake
  Extracted 1000 feature vectors

Processing class: ceviche


KeyboardInterrupt: 

In [ ]:
# Combine all features into single arrays
densenet_features_all = np.vstack([densenet_features_dict[cls] for cls in food_classes])
dinov2_features_all = np.vstack([dinov2_features_dict[cls] for cls in food_classes])
labels_array = np.array(labels)

print(f"\nDenseNet features shape: {densenet_features_all.shape}")
print(f"DinoV2 features shape: {dinov2_features_all.shape}")
print(f"Labels shape: {labels_array.shape}")

In [ ]:
output_dir = Path("data/features")
output_dir.mkdir(parents=True, exist_ok=True)

np.save(output_dir / "densenet_features.npy", densenet_features_all)
np.save(output_dir / "dinov2_features.npy", dinov2_features_all)
np.save(output_dir / "labels.npy", labels_array)

print(f"Features saved to {output_dir}/")

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler
import time

In [ ]:
# Scaling
scaler = StandardScaler()

densenet_features_scaled = scaler.fit_transform(densenet_features_all)
dinov2_features_scaled = scaler.fit_transform(dinov2_features_all)

### Apply Dimensionality Reduction Techniques

In [ ]:
reduced_features = {
    'densenet': {},
    'dinov2': {}
} # use dict so can extend other methods

# apply dimensionality reduction
def apply_dimensionality_reduction(features, **kwargs):
    start_time = time.time()
    reducer = umap.UMAP(n_components=2, **kwargs)
    
    reduced = reducer.fit_transform(features)
    elapsed_time = time.time() - start_time
    
    print(f"completed in {elapsed_time:.2f} seconds")
    print(f"Output shape: {reduced.shape}")
    
    return reduced, elapsed_time

print("Dimensionality reduction function defined")

In [ ]:
print("=" * 60)
print("DENSENET FEATURES - Dimensionality Reduction")
print("=" * 60)

# UMAP for DenseNet
reduced_features['densenet']['UMAP'], _ = apply_dimensionality_reduction(
    densenet_features_scaled, 'UMAP',
    random_state=42, n_neighbors=15, min_dist=0.1
)

print("\n" + "=" * 60)
print("DINOV2 FEATURES - Dimensionality Reduction")
print("=" * 60)

# UMAP for DinoV2
reduced_features['dinov2']['UMAP'], _ = apply_dimensionality_reduction(
    dinov2_features_scaled, 'UMAP',
    random_state=42, n_neighbors=15, min_dist=0.1
)

print("\n" + "=" * 60)
print("All dimensionality reduction complete!")
print("=" * 60)

### 2D Visualization of Reduced Features

In [ ]:
# Create color map for classes
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

# Create color palette
colors = plt.cm.tab10(np.linspace(0, 1, len(food_classes)))
class_to_color = {cls: colors[i] for i, cls in enumerate(food_classes)}
color_array = np.array([class_to_color[label] for label in labels_array])

# Create legend patches
legend_patches = [mpatches.Patch(color=class_to_color[cls], label=cls.replace('_', ' ')) 
                  for cls in food_classes]

print("Color mapping created for visualization")

### Individual Detailed Visualizations

In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# DenseNet + UMAP
ax = axes[0]
for i, cls in enumerate(food_classes):
    mask = labels_array == cls
    reduced = reduced_features['densenet']['UMAP']
    ax.scatter(
        reduced[mask, 0],
        reduced[mask, 1],
        c=[class_to_color[cls]],
        label=cls.replace('_', ' '),
        s=50,
        alpha=0.7,
        edgecolors='k',
        linewidth=0.5
    )

ax.set_title('DenseNet Features + UMAP', fontsize=16, fontweight='bold')
ax.set_xlabel('UMAP Component 1', fontsize=12)
ax.set_ylabel('UMAP Component 2', fontsize=12)
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

# DinoV2 + UMAP
ax = axes[1]
for i, cls in enumerate(food_classes):
    mask = labels_array == cls
    reduced = reduced_features['dinov2']['UMAP']
    ax.scatter(
        reduced[mask, 0],
        reduced[mask, 1],
        c=[class_to_color[cls]],
        label=cls.replace('_', ' '),
        s=50,
        alpha=0.7,
        edgecolors='k',
        linewidth=0.5
    )

ax.set_title('DinoV2 Features + UMAP', fontsize=16, fontweight='bold')
ax.set_xlabel('UMAP Component 1', fontsize=12)
ax.set_ylabel('UMAP Component 2', fontsize=12)
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('umap_detailed_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

### Quantitative Comparison of Methods

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

# Evaluate clustering quality using the reduced features
print("=" * 80)
print("CLUSTERING QUALITY METRICS (using class labels as ground truth)")
print("=" * 80)
print("\nHigher is better: Silhouette Score, Calinski-Harabasz Score")
print("Lower is better: Davies-Bouldin Score")
print("\n" + "-" * 80)

# Convert labels to numeric
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
numeric_labels = le.fit_transform(labels_array)

results = []

for model_name, model_label in zip(['densenet', 'dinov2'], ['DenseNet', 'DinoV2']):
    print(f"\n{model_label} Features:")
    print("-" * 80)
    
    for method in ['UMAP']:
        reduced = reduced_features[model_name][method]
        
        # Calculate metrics
        silhouette = silhouette_score(reduced, numeric_labels)
        davies_bouldin = davies_bouldin_score(reduced, numeric_labels)
        calinski = calinski_harabasz_score(reduced, numeric_labels)
        
        results.append({
            'Model': model_label,
            'Method': method,
            'Silhouette': silhouette,
            'Davies-Bouldin': davies_bouldin,
            'Calinski-Harabasz': calinski
        })
        
        print(f"\n  {method}:")
        print(f"    Silhouette Score:        {silhouette:.4f}")
        print(f"    Davies-Bouldin Score:    {davies_bouldin:.4f}")
        print(f"    Calinski-Harabasz Score: {calinski:.2f}")

print("\n" + "=" * 80)

In [ ]:
# Create a summary table
import pandas as pd

results_df = pd.DataFrame(results)
print("\nSummary Table:")
print(results_df.to_string(index=False))

methods = ['UMAP']

# Visualize metrics comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ['Silhouette', 'Davies-Bouldin', 'Calinski-Harabasz']
metric_labels = ['Silhouette Score\n(higher is better)', 
                 'Davies-Bouldin Score\n(lower is better)', 
                 'Calinski-Harabasz Score\n(higher is better)']

for idx, (metric, label) in enumerate(zip(metrics, metric_labels)):
    ax = axes[idx]
    
    # Prepare data for grouped bar chart
    x = np.arange(len(methods))
    width = 0.35
    
    densenet_values = results_df[results_df['Model'] == 'DenseNet'][metric].values
    dinov2_values = results_df[results_df['Model'] == 'DinoV2'][metric].values
    
    ax.bar(x - width/2, densenet_values, width, label='DenseNet', alpha=0.8)
    ax.bar(x + width/2, dinov2_values, width, label='DinoV2', alpha=0.8)
    
    ax.set_xlabel('Dimensionality Reduction Method', fontsize=11)
    ax.set_ylabel(label, fontsize=11)
    ax.set_title(f'{metric} Comparison', fontsize=13, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(methods)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('dimensionality_reduction_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Save the reduced features for later use in clustering
np.save(output_dir / "densenet_pca.npy", reduced_features['densenet']['PCA'])
np.save(output_dir / "densenet_tsne.npy", reduced_features['densenet']['t-SNE'])
np.save(output_dir / "densenet_umap.npy", reduced_features['densenet']['UMAP'])

np.save(output_dir / "dinov2_pca.npy", reduced_features['dinov2']['PCA'])
np.save(output_dir / "dinov2_tsne.npy", reduced_features['dinov2']['t-SNE'])
np.save(output_dir / "dinov2_umap.npy", reduced_features['dinov2']['UMAP'])

print(f"\nAll reduced features saved to {output_dir}/")